In [ ]:
import geopandas as gpd
import pandas as pd
from gtfs_loader import load_transit_dataframe

transit_df = load_transit_dataframe("gtfs")

def openMap(m):
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [ ]:
# create geopandas from transit2 using shape_geometry column
gdf_transit = gpd.GeoDataFrame(transit_df, geometry='stop_geometry', crs="EPSG:4326")
m = gdf_transit.explore()

In [ ]:
#read stops.txt
stop_df = pd.read_csv("gtfs/stops.txt")
stop_df.info()
#filter stops to only those in transit_df["stop_ids"]
stop_df = stop_df[stop_df['stop_id'].isin(transit_df['stop_ids'].explode().unique())]



In [ ]:
from collections import defaultdict
import networkx as nx

# Use defaultdict to map stop_id to sets
next_stop_dict = defaultdict(dict)
routes_dict = defaultdict(set)

for stops_per_trip, stop_time_deltas, shape_id, route_id in zip(transit_df['stop_ids'], transit_df['stop_time_deltas'], transit_df['shape_id'], transit_df['route_id']):
    for i in range(len(stops_per_trip)-1):
        stop_id = stops_per_trip[i]
        next_stop_id = stops_per_trip[i+1]
        stop_time_delta = stop_time_deltas[i]
        # if empty key, assign stop_time_delta and shape_id in a list
        if next_stop_id not in next_stop_dict[stop_id]:
            next_stop_dict[stop_id][next_stop_id] = {'weight': stop_time_delta, 'shape_ids': [shape_id]}
        # otherwise append shape_id to the list and update the average stop_time_delta            
        else:
            next_stop_dict[stop_id][next_stop_id]['shape_ids'].append(shape_id)
            next_stop_dict[stop_id][next_stop_id]['weight'] = round((next_stop_dict[stop_id][next_stop_id]['weight'] + stop_time_delta) / (len(next_stop_dict[stop_id][next_stop_id]['shape_ids'])))
         
        routes_dict[stop_id].add(route_id)

stop_df['next_stop_id'] = stop_df['stop_id'].map(lambda x: next_stop_dict.get(x, dict()))
stop_df['routes_by_stop'] = stop_df['stop_id'].map(lambda x: routes_dict.get(x, set()))

#remove stops with no next_stop_id
stop_df = stop_df[stop_df['next_stop_id'].map(len) > 0]

In [ ]:
# stops_gdf
stops_gdf = gpd.GeoDataFrame(stop_df, geometry=gpd.points_from_xy(stop_df['stop_lon'], stop_df['stop_lat']), crs="EPSG:4326")

#change its crs to metric
#stops_gdf = stops_gdf.to_crs(epsg=3857)

#each point convert it to a circle of radius 50 m
#stops_gdf['geometry'] = stops_gdf.buffer(50)

#change its crs to geographic
stops_gdf = stops_gdf.to_crs(epsg=4326)

In [ ]:
#create graph from next_stop_dict
G = nx.DiGraph()
for stop_id, next_stops in next_stop_dict.items():
    for next_stop_id, attr in next_stops.items():
        G.add_edge(stop_id, next_stop_id, **attr)


#choose randomly a node from G
import random

path = []

while path == []:
    start = random.choice(list(G.nodes))
    destination = random.choice(list(G.nodes))

    try:
        path = nx.shortest_path(G, source=start, target=destination, weight='weight')
    except nx.NetworkXNoPath:
        path = []

print("Shortest path from", start, "to", destination, ":", path)

path_gdf = stops_gdf[stops_gdf['stop_id'].isin(path)]
m = path_gdf.explore()
openMap(m)

G.nodes[path[0]]

In [ ]:

	x
# create osmnx graph from G
import pandas as pd
import osmnx as ox

# Create edge DataFrame from networkx DiGraph
edgeslist_df = nx.to_pandas_edgelist(G)
edgeslist_df = edgeslist_df.rename(columns={'source': 'stop_id'})

# add column x and y from stops_df lon and lat
edgeslist_df = edgeslist_df.merge(
	stop_df[['stop_id', 'stop_lon', 'stop_lat']].rename(columns={'stop_lon': 'x', 'stop_lat': 'y'}),
	left_on='stop_id',
	right_on='stop_id'
)


edgeslist_df['geometry'] = gpd.points_from_xy(edgeslist_df['x'], edgeslist_df['y'], crs="EPSG:4326")

# Rename columns to match OSMnx expectations
edgeslist_df = edgeslist_df.rename(columns={'stop_id': 'osmid'})

# Create GeoDataFrame for edges
nodes_gdf = gpd.GeoDataFrame(edgeslist_df[['osmid', 'x', 'y', 'geometry']], geometry='geometry', crs="EPSG:4326")

edgeslist_df = edgeslist_df.rename(columns={'osmid': 'u', 'target': 'v', 'weight': 'length', 'geometry': 'point_geometry'})

# Create new column geometry as LineString from point_geometry of u and v
from shapely.geometry import LineString
def create_linestring(row):
	point_u = row['point_geometry']
	# Use .loc[row, col] for single-level index
	idx = stops_gdf[stops_gdf['stop_id'] == row['v']]['geometry'].values
	if len(idx) == 0:
		return None  # or point_u if you want a degenerate line
	point_v = idx[0]
	return LineString([point_u, point_v])

edgeslist_df['geometry'] = edgeslist_df.apply(create_linestring, axis=1)

edges_gdf = gpd.GeoDataFrame(edgeslist_df[['u', 'v', 'geometry', 'length']], geometry='geometry', crs="EPSG:4326")

G_osm = ox.graph_from_gdfs(nodes_gdf,	edges_gdf)

In [ ]:
# I want to add all the shape_id that intersect the circles in a new column called shape_ids

#stops_gdf['shape_ids'] = None
#for index, row in stops_gdf.iterrows():
#    intersecting_shapes = []
#    for shape_id, shape_geom in zip(transit_df['shape_id'], transit_df['shape_geometry']):
#        if row['geometry'].intersects(shape_geom):
#            intersecting_shapes.append(shape_id)
#    stops_gdf.at[index, 'shape_ids'] = intersecting_shapes

In [ ]:
# stops gdf explore in a new layer as red circles size 4
m = stops_gdf.explore( color='red', style_kwds={'fillOpacity': 0.05, 'weight': 1})

In [ ]:
# create the GeoDataFrame from shape_geometry column in the transit_df
gdf_shapes = gpd.GeoDataFrame(transit_df, geometry='shape_geometry', crs="EPSG:4326")
gdf_shapes.explore(m=m)